# **SDSE Final Project: Predicting Car Fuel Efficiency with Machine Learning**

Our notebook attempts to predict a given car's **combined fuel efficiency (MPG)** based on information that is available **before** buying the car:

- Make and type                 (SUV, sedan, etc.)
- Drivetrain                    (FWD, AWD, etc.)
- Engine displacement           (L)
- Number of cylinders           (dimensinoless)
- Transmission type             (automatic, manual, etc.)

Why this matters:

- Consumers can estimate real-world fuel costs for a car model even when independent test results are not widely available yet, such with new vehicle models and trims.
- Dealerships and fleet managers can compare many options quickly based on expected efficiency.
- Policy or sustainability teams can more immediately simulate how changing the mix of vehicles (more small engines, more hybrids, etc.) might affect fuel consumption and greenhouse/noise emissions.

The notebook follows the outline specified for the project submissions in the lab session. 

0. Choose performance metric
1. load data
2. split off the test data set
3. Choose the families of model and hyperparameter variations to test.
4. Execute (train and evaluate) models
5. Choose best model and evaluate with test data.

# 0. Performance metric:

Since we are working with a regression model, we will be using the R^2 coefficient of determination as the metric to evaluate all our models.

## 1. Load Data
In this section we:


- **1.1**: import necessary libraries/packages
    - **pandas / numpy** for data manipulation  
    - **matplotlib** for plotting  
    - **scikit‑learn** tools for preprocessing and linear regression  
    - **TensorFlow / Keras** for building neural networks
- read in the data
- clean up the data
- plot data
- reduce number of classes in categories where there are too many (consolidation)





In [40]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization


In [41]:
# Reading in the data
data = pd.read_csv("car_data.csv")
# print(First 5 rows of data:")
# print(data.head(5))

**1.3: Defining Feature Columns and Target**

Next, we decide which columns will be used as inputs (features) and which column is the output (target).

- `categorical` lists all string‑based variables we might want to explore.
- `numerical` contains the numeric engine characteristics that are important for fuel use.
- `output` is the target: combined MPG.
- `categorical_for_model` is a refined list of categorical columns used in the actual model.  
  We keep a fixed set of five categorical variables as required by the project guidelines, and avoid overly specific identifiers which would be difficult to generalize well, such as the model name.

In [42]:
# Feature lists
categorical = ['type', 'drive', 'make', 'model', 'transmission']
numerical = ['cylinders', 'displacement']
output = 'combination_mpg'
categorical_for_model = ['type', 'drive', 'fuel_type', 'make', 'transmission']

**1.4: Data Cleaning and Preparation**

Before training any model, our code handles missing values:

1. **Drop rows with missing target**:  
   If `combination_mpg` is missing, the data is unusable for supervised learning.

2. **Impute numerical features with the mean**:  
   This maintains a baseline that keeps all existing data while avoiding bias towards any existing value.

3. **Impute categorical features with the mode** (most frequent category):  
   This preserves the most likely class and keeps categories consistent.

After cleaning, our code builds:

- `feature_cols`: all columns that will be used as inputs
- `X`: the feature matrix  
- `y`: the target vector

In [43]:
# Data preparation
data = data.dropna(subset=[output])

for col in numerical:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].mean())

for col in categorical_for_model:
    if col in data.columns:
        data[col] = data[col].fillna(data[col].mode()[0])

# # Defining feature matrix X and target y
# feature_cols = categorical_for_model + numerical
# X = data[feature_cols]
# y = data[output]

# print("\nFeature columns used for the model:")
# print(feature_cols)
# print(f"\nNumber of samples after cleaning: {len(X)}")

**1.5: Consolidating categories with too many classes**

We will consolidate the categories "make" and "type". We will ignore the "model" data in the model since each entry is a different model.

The "make" category will be reduced to 3 classes by region of the company:
1. Asia: kia, hyundai, genesis, mazda, honda, acura, subaru, mitsubishi, toyota, nissan, infiniti
2. Europe: bmw, jaguar, mini, audi, land rover, volvo, volkswagen, aston martin, porsche, bentley, mercedes-benz
3. America: chevrolet, jeep, gmc, ford, cadillac, buick, ram, roush performance, chrysler

The "type" category will be reduced to 3 classes by size:
1. small: small sport utility vehicle, subcompact car, compact car, two seater, minicompact car, small station wagon
2. medium: midsize car, standard sport utility vehicle, small pickup truck, midsize station wagon
3. large: large car, minivan, standard pickup truck


In [44]:
# defining region lists for make region categories
asia = ["kia", "hyundai", "genesis", "mazda", "honda", "acura",
    "subaru", "mitsubishi", "toyota", "nissan", "infiniti"]

europe = ["bmw", "jaguar", "mini", "audi", "land rover", "volvo",
    "volkswagen", "aston martin", "porsche", "bentley", "mercedes-benz"]

america = ["chevrolet", "jeep", "gmc", "ford", "cadillac", "buick",
    "ram", "roush performance", "chrysler"]

def consolidate_region(make):
    if make in asia:
        return "asia"
    if make in europe:
        return "europe"
    if make in america:
        return "america"
    return "other"   # just in case

# consolidating make data to make_region
data["make_region"] = data["make"].apply(consolidate_region)

small = ["small sport utility vehicle","subcompact car","compact car",
            "two seater","minicompact car","small station wagon"]
medium = ["midsize car", "standard sport utility vehicle", 
            "small pickup truck", "midsize station wagon"]
large = ["large car", "minivan", "standard pickup truck"]

def consolidate_size(type):
    if type in small:
        return "small"
    if type in medium:
        return "medium"
    if type in large:
        return "large"
    return "other"
        
# consolidating type data to size
data["size"] = data["type"].apply(consolidate_size)


**1.X: spomething**



In [45]:
categorical_for_model_consolidated = ['size', 'drive', 'fuel_type', 'make_region', 'transmission']
# Defining feature matrix X and target y
feature_cols = categorical_for_model_consolidated + numerical
X = data[feature_cols]
y = data[output]

print("\nFeature columns used for the model:")
print(feature_cols)
print(f"\nNumber of samples after cleaning: {len(X)}")


Feature columns used for the model:
['size', 'drive', 'fuel_type', 'make_region', 'transmission', 'cylinders', 'displacement']

Number of samples after cleaning: 550


## 2. Split off the test dataset:

In [46]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"\nTrain size: {X_train.shape[0]} samples")
print(f"Test size: {X_test.shape[0]} samples")


Train size: 440 samples
Test size: 110 samples


**2.X: One-hot encoding**
Machine learning models like linear regression work with numbers, not strings.  
We therefore need to convert categorical features into a numeric format.

We use:

- **`ColumnTransformer` with `OneHotEncoder`**  
  - Each categorical column is expanded into one binary column per category.
  - `handle_unknown='ignore'` ensures the model can handle categories that only appear in the test set.

In [48]:
# One-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_for_model_consolidated)
    ],
    remainder='passthrough'  # keep numerical columns as-is
)

# 1. Fit preprocessor ONLY on training data
preprocessor.fit(X_train)

# 2. Transform train and test sets
X_train_enc = preprocessor.transform(X_train)
X_test_enc  = preprocessor.transform(X_test)

